In [1]:
import numpy as np
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
import torch
from torch.nn import functional as F

# vector

In [2]:
vector1 = np.array([1., 1])
vector2 = np.array([-1., 1])

## cosine_similarity(vector1, vector2) = (vector1 / l2norm(vector1)) @ (vector2 / l2norm(vector2)).T

## numpy

In [3]:
np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

0.0

In [4]:
vector1 @ vector2.T / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

0.0

In [5]:
vector1 @ vector2.T / (np.sqrt(np.sum(vector1 ** 2)) * np.sqrt(np.sum(vector2 ** 2)))

0.0

In [6]:
# 先除以各自的norm再计算矩阵相乘
(vector1 / np.linalg.norm(vector1)) @ (vector2 / np.linalg.norm(vector2)).T

0.0

## scipy

In [7]:
1 - spatial.distance.cosine(vector1, vector2)

0.0

## sklearn

In [8]:
vector1.reshape(1, -1), vector2.reshape(1, -1)

(array([[1., 1.]]), array([[-1.,  1.]]))

In [9]:
cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))

array([[0.]])

## pytorch

In [10]:
tensor1 = torch.from_numpy(vector1)
tensor2 = torch.from_numpy(vector2)
tensor1, tensor2

(tensor([1., 1.], dtype=torch.float64),
 tensor([-1.,  1.], dtype=torch.float64))

In [11]:
F.cosine_similarity(tensor1, tensor2, dim=0)

tensor(0., dtype=torch.float64)

In [12]:
torch.dot(tensor1, tensor2) / (tensor1.norm() * tensor2.norm())

tensor(0., dtype=torch.float64)

In [13]:
tensor1 @ tensor2.T / (tensor1.norm() * tensor2.norm())

C:\Users\Administrator\AppData\Local\Temp\ipykernel_60820\4149563073.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3641.)
  tensor1 @ tensor2.T / (tensor1.norm() * tensor2.norm())


tensor(0., dtype=torch.float64)

In [14]:
# 先除以各自的norm再计算矩阵相乘
(tensor1 / tensor1.norm()) @ (tensor2 / tensor2.norm()).T

tensor(0., dtype=torch.float64)

# Matrix

## cosine_similarity(matrix1, matrix2) = (matrix1 / l2norm(matrix1)) @ (matrix2 / l2norm(matrix2)).T

In [16]:
matrix1 = np.array([[1., 1],
                    [1., 0]])
matrix2 = np.array([[-1., 1],
                    [2., 1],
                    [-1., -1]])
# cos value     matrix2[0]  matrix2[1]  matrix2[2]
# matrix1[0]    0           0.9487      -1
# matrix1[1]    -0.7071     0.8944      -0.7071

## numpy

In [18]:
print(np.linalg.norm(matrix1, axis=-1, keepdims=True))
print(np.linalg.norm(matrix2, axis=-1, keepdims=True).T)

[[1.41421356]
 [1.        ]]
[[1.41421356 2.23606798 1.41421356]]


In [19]:
np.linalg.norm(matrix1, axis=-1, keepdims=True) @ np.linalg.norm(matrix2, axis=-1, keepdims=True).T # 这里用 * 或者 @ 结果相同, * 使用了广播机制

array([[2.        , 3.16227766, 2.        ],
       [1.41421356, 2.23606798, 1.41421356]])

In [20]:
matrix1 @ matrix2.T / (np.linalg.norm(matrix1, axis=-1, keepdims=True) @ np.linalg.norm(matrix2, axis=-1, keepdims=True).T)

array([[ 0.        ,  0.9486833 , -1.        ],
       [-0.70710678,  0.89442719, -0.70710678]])

In [21]:
# 先除以各自的norm再计算矩阵相乘
(matrix1 / np.linalg.norm(matrix1, axis=-1, keepdims=True)) @ (matrix2 / np.linalg.norm(matrix2, axis=-1, keepdims=True)).T

array([[ 2.23711432e-17,  9.48683298e-01, -1.00000000e+00],
       [-7.07106781e-01,  8.94427191e-01, -7.07106781e-01]])

## sklearn

In [22]:
cosine_similarity(matrix1, matrix2)

array([[ 2.23711432e-17,  9.48683298e-01, -1.00000000e+00],
       [-7.07106781e-01,  8.94427191e-01, -7.07106781e-01]])

## pytorch

In [23]:
tensor1 = torch.from_numpy(matrix1)
tensor2 = torch.from_numpy(matrix2)
tensor1, tensor2

(tensor([[1., 1.],
         [1., 0.]], dtype=torch.float64),
 tensor([[-1.,  1.],
         [ 2.,  1.],
         [-1., -1.]], dtype=torch.float64))

In [24]:
print(tensor1.unsqueeze(0))
print(tensor2.unsqueeze(1))

tensor([[[1., 1.],
         [1., 0.]]], dtype=torch.float64)
tensor([[[-1.,  1.]],

        [[ 2.,  1.]],

        [[-1., -1.]]], dtype=torch.float64)


In [25]:
# 要添加维度并转置结果
F.cosine_similarity(tensor1.unsqueeze(0), tensor2.unsqueeze(1), dim=-1).T

tensor([[ 0.0000,  0.9487, -1.0000],
        [-0.7071,  0.8944, -0.7071]], dtype=torch.float64)

In [26]:
tensor1.norm(p=2, dim=-1, keepdim=True) @ tensor2.norm(p=2, dim=-1, keepdim=True).T # 这里用 * 或者 @ 结果相同, * 使用了广播机制

tensor([[2.0000, 3.1623, 2.0000],
        [1.4142, 2.2361, 1.4142]], dtype=torch.float64)

In [27]:
# 对于多维多来说每个值都要除以对应vector的norm的乘积
tensor1 @ tensor2.T / (tensor1.norm(p=2, dim=-1, keepdim=True) @ tensor2.norm(p=2, dim=-1, keepdim=True).T)

tensor([[ 0.0000,  0.9487, -1.0000],
        [-0.7071,  0.8944, -0.7071]], dtype=torch.float64)

In [28]:
# 可以先除以各自的norm再计算矩阵相乘
(tensor1 / (tensor1.norm(p=2, dim=-1, keepdim=True))) @ (tensor2 / tensor2.norm(p=2, dim=-1, keepdim=True)).T

tensor([[ 0.0000,  0.9487, -1.0000],
        [-0.7071,  0.8944, -0.7071]], dtype=torch.float64)

# Vector&Matrix

In [29]:
vector1 = np.array([1., 1])
matrix1 = np.array([[-1.,  1],
                    [2.,   1],
                    [-1., -1]])
# cos value     matrix2[0]  matrix2[1]  matrix2[2]
# vector1       0           0.9487      -1

## numpy

In [30]:
(np.linalg.norm(vector1, axis=-1, keepdims=True) @ np.linalg.norm(matrix1, axis=-1, keepdims=True).T) # 这里用 * 或者 @ 结果相同, * 使用了广播机制

array([2.        , 3.16227766, 2.        ])

In [31]:
vector1 @ matrix1.T / (np.linalg.norm(vector1, axis=-1, keepdims=True) @ np.linalg.norm(matrix1, axis=-1, keepdims=True).T)

array([ 0.       ,  0.9486833, -1.       ])

In [32]:
# 可以先除以各自的norm再计算矩阵相乘
(vector1 / np.linalg.norm(vector1, axis=-1, keepdims=True)) @ (matrix1 / np.linalg.norm(matrix1, axis=-1, keepdims=True)).T

array([-2.23711432e-17,  9.48683298e-01, -1.00000000e+00])

In [33]:
# 颠倒前后顺序
(matrix1 / np.linalg.norm(matrix1, axis=-1, keepdims=True)) @ (vector1 / np.linalg.norm(vector1, axis=-1, keepdims=True)).T

array([-2.23711432e-17,  9.48683298e-01, -1.00000000e+00])

## sklearn

In [34]:
cosine_similarity(vector1.reshape(1, -1), matrix1)

array([[-2.23711432e-17,  9.48683298e-01, -1.00000000e+00]])

In [35]:
# 颠倒前后顺序
cosine_similarity(matrix1, vector1.reshape(1, -1))

array([[-2.23711432e-17],
       [ 9.48683298e-01],
       [-1.00000000e+00]])

## pytorch

In [36]:
tensor1 = torch.from_numpy(vector1)
tensor2 = torch.from_numpy(matrix1)

In [37]:
# 要添加维度并转置结果
F.cosine_similarity(tensor1.unsqueeze(0), tensor2, dim=-1).T

tensor([ 0.0000,  0.9487, -1.0000], dtype=torch.float64)

In [38]:
tensor1.norm(p=2, dim=-1, keepdim=True) @ tensor2.norm(p=2, dim=-1, keepdim=True).T # 这里用 * 或者 @ 结果相同, * 使用了广播机制

tensor([2.0000, 3.1623, 2.0000], dtype=torch.float64)

In [39]:
tensor1 @ tensor2.T / (tensor1.norm(dim=-1, p=2, keepdim=True) @ tensor2.norm(p=2, dim=-1, keepdim=True).T)

tensor([ 0.0000,  0.9487, -1.0000], dtype=torch.float64)

In [40]:
# 可以先除以各自的norm再计算矩阵相乘
tensor1 / (tensor1.norm(dim=-1, p=2, keepdim=True)) @ (tensor2 / tensor2.norm(p=2, dim=-1, keepdim=True)).T

tensor([ 0.0000,  0.9487, -1.0000], dtype=torch.float64)

In [41]:
# 颠倒前后顺序
(tensor2 / tensor2.norm(p=2, dim=-1, keepdim=True)) @ (tensor1 / tensor1.norm(p=2, dim=-1, keepdim=True)).T

tensor([ 0.0000,  0.9487, -1.0000], dtype=torch.float64)